In [1]:
# Import necessary libraries
import pandas as pd               
import numpy as np                
import matplotlib.pyplot as plt   
import os                         
import joblib                     
import hopsworks                  
import re

# Import specific modules from scikit-learn
from sklearn.preprocessing import StandardScaler, OneHotEncoder   # For data preprocessing
from sklearn.metrics import accuracy_score                        # For evaluating model accuracy

from dotenv import load_dotenv
import os
load_dotenv()

#Connecting to hopsworks
api_key = os.environ.get('HOPSWORKS_API')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/564374
2024-10-13 10:47:43,148 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.


In [2]:
# Load and display the data from CSV to get an overview of the actual data - confirming the values
amd_df = pd.read_csv('AMD_stock_prices.csv')
print(amd_df.head())    

         date  1. open   2. high    3. low  4. close   5. volume ticker
0  2024-10-04  166.533  171.2100  164.4200    170.90  44640296.0    AMD
1  2024-10-03  159.710  166.7894  159.4000    162.85  29598787.0    AMD
2  2024-10-02  159.310  163.1000  158.6689    159.78  24630431.0    AMD
3  2024-10-01  164.480  165.4000  158.0800    159.75  31831860.0    AMD
4  2024-09-30  163.100  165.0800  162.1800    164.08  21920300.0    AMD


In [3]:
amd_df

,date,1. open,2. high,3. low,4. close,5. volume,ticker
0,2024-10-04,166.533,171.2100,164.4200,170.90,44640296.0,AMD
1,2024-10-03,159.710,166.7894,159.4000,162.85,29598787.0,AMD
2,2024-10-02,159.310,163.1000,158.6689,159.78,24630431.0,AMD
3,2024-10-01,164.480,165.4000,158.0800,159.75,31831860.0,AMD
4,2024-09-30,163.100,165.0800,162.1800,164.08,21920300.0,AMD
...,...,...,...,...,...,...,...
6267,1999-11-05,20.750,21.3800,20.2500,21.25,2148000.0,AMD
6268,1999-11-04,21.130,21.3100,19.9400,20.63,2415000.0,AMD
6269,1999-11-03,20.690,21.5000,20.5600,21.31,2236000.0,AMD
6270,1999-11-02,20.310,20.8100,20.3100,20.56,1898100.0,AMD


In [4]:
# Converting the "date" column to timestamp
amd_df['date'] = pd.to_datetime(amd_df['date'])

In [5]:
import re

# Function to clean column names
def clean_column_name(col):
    # Convert to lowercase
    col = col.lower()
    # Replace any invalid characters with an underscore
    col = re.sub(r'[^a-z0-9_]', '_', col)
    # If the column name doesn't start with a letter, prefix it with 'f_'
    if not re.match(r'^[a-z]', col):
        col = 'f_' + col
    return col

# Apply the function to all column names
amd_df.columns = [clean_column_name(col) for col in amd_df.columns]

print("Cleaned column names:")
print(amd_df.columns.tolist())


Cleaned column names:
['date', 'f_1__open', 'f_2__high', 'f_3__low', 'f_4__close', 'f_5__volume', 'ticker']


In [7]:
# version 7 for the new feature group
amd_fg = fs.create_feature_group(
    name="amd_stock",
    description="AMD stock dataset from Alpha Vantage",
    version=8,  # Use the new version number
    primary_key=["ticker"],
    event_time="date",
    online_enabled=False,
)

# Insert data into the new feature group
#amd_fg.insert(amd_df, write_options={"wait_for_job": False})
try:
    amd_fg.insert(amd_df, write_options={"wait_for_job": False})
    print("\nData inserted successfully into the feature group.")
except Exception as e:
    print(f"\nError inserting data into feature group: {e}")


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/564374/fs/560197/fg/1273172


Uploading Dataframe: 0.00% |          | Rows 0/6272 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: amd_stock_8_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/564374/jobs/named/amd_stock_8_offline_fg_materialization/executions

Data inserted successfully into the feature group.
